In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append(r'/home/gabriel/Documents/proj/utils/')

from utils import createSparkSesion
from pyspark.sql import (functions as F,
                                        Window)
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
spark = createSparkSesion()

25/04/03 21:09:02 WARN Utils: Your hostname, GabrielUnbunto resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/04/03 21:09:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/03 21:09:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark.sparkContext

<SparkContext master=local[*] appName=Analises>

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/[location]/[date1]/[date2]?key=YOUR_API_KEY

In [5]:
api_base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
api_key      = "PQWYU6EJ2B473ZQS6SASXUJAZ"

In [6]:
start_date  = (datetime.today()).strftime('%Y-%m-%d')
end_date    = (datetime.today() + timedelta(days=7)).strftime('%Y-%m-%d')

In [7]:
location = 'Boston'

In [8]:
request_url = f'{api_base_url}/{location}/{start_date}/{end_date}?unitGroup=metric&include=days&key={api_key}&contentType=csv'
request_url

'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline//Boston/2025-04-03/2025-04-10?unitGroup=metric&include=days&key=PQWYU6EJ2B473ZQS6SASXUJAZ&contentType=csv'

In [9]:
pd_weather_df  = pd.read_csv(request_url)

In [22]:
weather_df = spark.createDataFrame(pd_weather_df).withColumn("inference_date" , F.lit(datetime.now().strftime("%Y-%m-%d")))

In [23]:
weather_df.takePandas()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations,inference_date
0,"Boston, MA, United States",2025-04-03,17.9,2.3,9.9,17.9,-2.4,7.9,7.8,87.3,...,8,10,2025-04-03T06:22:44,2025-04-03T19:12:50,0.18,"Rain, Overcast",Cloudy skies throughout the day with a chance ...,rain,"KOWD,AV085,KBED,0518W,KBOS",2025-04-03
1,"Boston, MA, United States",2025-04-04,15.0,9.5,12.9,15.0,7.8,12.9,4.7,58.8,...,7,10,2025-04-04T06:21:02,2025-04-04T19:13:57,0.25,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,NaN,2025-04-03
2,"Boston, MA, United States",2025-04-05,8.9,6.2,6.6,7.1,2.7,3.6,0.8,66.6,...,2,10,2025-04-05T06:19:20,2025-04-05T19:15:05,0.25,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,NaN,2025-04-03
3,"Boston, MA, United States",2025-04-06,12.8,6.6,9.9,12.8,3.6,8.7,7.6,86.4,...,0,10,2025-04-06T06:17:39,2025-04-06T19:16:13,0.28,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,NaN,2025-04-03
4,"Boston, MA, United States",2025-04-07,10.0,5.4,7.3,8.3,3.7,5.3,0.9,64.1,...,0,10,2025-04-07T06:15:58,2025-04-07T19:17:20,0.32,"Rain, Partially cloudy",Partly cloudy throughout the day with rain cle...,rain,NaN,2025-04-03


In [ ]:
(weather_df.write 
    .format("jdbc") 
    .option("url", "jdbc:postgresql://localhost:5432/weatherProject") 
    .option("dbtable", "raw_weather_info") 
    .option("user", "postgres") 
    .option("password", "30061725") 
    .option("driver", "org.postgresql.Driver") 
    .mode("append")   
    .save())
